<a href="https://colab.research.google.com/github/manthanawgan/ML-Algorithms/blob/main/Locally_Weighted_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import operator
from os import listdir
from numpy.linalg import *
from scipy.stats.stats import pearsonr


<ipython-input-1-2ffe25d96b49>:7: DeprecationWarning: Please import `pearsonr` from the `scipy.stats` namespace; the `scipy.stats.stats` namespace is deprecated and will be removed in SciPy 2.0.0.
  from scipy.stats.stats import pearsonr


**Preapare the data**

In [2]:
df = pd.read_csv('/content/tips.csv')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [6]:
features = np.array(df.total_bill)
labels = np.array(df.tip)
m = features.shape[0]
mtip = np.mat(labels)
data = np.hstack((np.ones((m, 1)), np.mat(features).T))

**Implementation**

In [11]:
def kernel(data, point, xmat, k):  #k is the width of the kernel
  m, n = np.shape(xmat)
  ws = np.mat(np.eye((m)))
  for j in range(m):
    diff = point - data[j]
    ws[j,j] = np.exp(diff*diff.T/(-2.0*k**2))
  return ws

def local_weight(data, point, xmat, ymat, k):
  weight = kernel(data, point, xmat, k)
  return(data.T*(weight*data)).I*(data.T(weight*ymat.T))

def local_weight_regression(xmat, ymat, k):
  m,n = np.shape(xmat)
  ypredictn = np.zeros(m)
  for i in range(m):
    ypredictn[i] = xmat[i]*local_weight(xmat, xmat[i], xmat, ymat, k)
  return ypredictn



In [12]:
ypredictn = local_weight_regression(data, mtip, 0.5)
indices = data[:,1].argsort(0)
xsort = data[indices][:,0]

<ipython-input-11-83ad394ee9dc>:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  ws[j,j] = np.exp(diff*diff.T/(-2.0*k**2))


TypeError: 'matrix' object is not callable